<a href="https://colab.research.google.com/github/brypergolic93/greatcoursesplus/blob/main/ch_03_decision_trees_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import tree
import graphviz
import random
import matplotlib.pyplot as plot

In [2]:
!wget https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv

--2021-03-09 06:01:48--  https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23873 (23K) [text/plain]
Saving to: ‘diabetes.csv’

diabetes.csv        100%[===================>]  23.31K  --.-KB/s    in 0.002s  

2021-03-09 06:01:48 (11.0 MB/s) - ‘diabetes.csv’ saved [23873/23873]



In [13]:
with open("diabetes.csv", "r") as f:
    data = f.readlines()
feats = data[0]
feats = feats.replace('\n','')
feats = feats.split(",") 

# the last column are the labels ("Outcomes") so we will remove that column from the features
feats = feats[0:(len(feats) - 1)]

In [20]:
data_update = []
labels = []

for i in range (1,len(data)):

    line = data[i]
    line = line.replace('\n','')
    csvline = line.split(",")

    labels += [int(csvline[len(csvline) - 1])]
    csvline = [float(csvline[i]) for i in range(len(csvline)-1)]
    data_update += [csvline]

[]
